In [5]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [11]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920,1080))
display.start()

In [12]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# genetic
genetic_path = '//*[@id="QU.SE.7.6-genetic-toxicity-header"]/div/div[2]'
driver.find_element_by_xpath(genetic_path).click()
time.sleep(0.5)

# header 하위 3개의 창
#Genetic vivo
genetic_vivo_path = '//*[@id="QU.SE.7.6-genetic-toxicity"]/div/div/div[2]/div[3]/button'
driver.find_element_by_xpath(genetic_vivo_path).click()
time.sleep(0.5)

/tmp/ipykernel_5948/2490923882.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [13]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG475경로
tg475_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[31]'
driver.find_element_by_xpath(tg475_path).click()
time.sleep(0.5)

#TG475경로 여기는 30개인가
tg475_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[31]'
driver.find_element_by_xpath(tg475_path).click()
time.sleep(0.5)

#tg475 여기가 700개가량
tg475_2path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[32]'
driver.find_element_by_xpath(tg475_2path).click()
time.sleep(0.5)

#OPPTS EPA8705385
epa8705385_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[6]'
driver.find_element_by_xpath(epa8705385_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [14]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [15]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(1)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
            #driver.find_element_by_xpath(accept_path).click()
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg475_source.json', 'w'))

100%|███████████████████████████████████████████████████████████████████████████| 3/3 [00:06<00:00,  2.18s/it, page=148]
2478.8089377880096


In [ ]:
##json전처리

In [2]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg475_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []

for i in tqdm(range(len(df))):
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        
        # Metabolic activation
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Metabolic activation:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Metabolic activation'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Metabolic activation'] = np.nan
            
        # Genotoxicity
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sResultsAndDiscussion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Genotoxicity:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['Genotoxicity'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['Genotoxicity'] = np.nan
            
            
            
        # Body weight and weight changes
        try:
            admin_tmp = soup.find('h3', attrs = {'id': 'sApplicantSummaryAndConclusion'})
            
            admin_tmp = admin_tmp.find_next_siblings('dl')
            admin_idx = [remove_bracket(x.find_next('dt')) for x in admin_tmp].index('Conclusions:')  # 결과 확인
            
            route_tmp = admin_tmp[admin_idx]
            
            chem_dict['conclusion'] = remove_bracket(route_tmp.find_next('dd'))
            
        except ValueError:
            chem_dict['conclusion'] = np.nan
            
            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        pass
    


# %%
result = pd.DataFrame(result_)

100%|█████████████████████████████████████████████████████████████████████████████████| 738/738 [00:22<00:00, 33.10it/s]


In [3]:
result['Genotoxicity'].value_counts()

negative                                                                                                                                                                                                                                                                654
positive                                                                                                                                                                                                                                                                 46
ambiguous                                                                                                                                                                                                                                                                20
other: no conclusive statement can be made                                                                                                                                                          

In [11]:

#%%
'''
    1. nan이 아닌 경우
    aerosol -> 보수적인 쪽으로. Dusts and Mists로 넣으면 됨 (aerosol include mists, smokes, fumes, and dusts)
    
    vapour + aerosol 같이 여러개가 혼합된 경우 보수적인 카테고리로 포함
    
    aerosol, vapour, gas, dust가 포함되는 애들 찾고
    aerosol이 포함되면 aerosol category
    aerosol 없이 vapour or vapor or vaporization 포함되면 vapour
    aerosol 없이 gas 포함되면 gas
    
    2. nan인경우 많은 쪽으로 포함
    3. inhalation만 있는 경우 많은 쪽으로 포함
    4. aerosol, vapour, gas, dust, mist가 포함되어있지 않으면 많은 쪽으로 포함
    
    1. effects observed가 하나라도 있으면 1
    1-1. no effects observed가 있으면 0
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'negative' in string:
            type_ = 0
        
        elif 'positive' in string:
            type_ = 1
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

def conclusion(string):
    if check_nan(string):
        string = string.lower()
        #주요 단어를 적는 곳인듯
        if 'no effect' in string or 'no observed' in string or 'no evidence' in string or 'No reproductive toxicity' in string \
        or 'no reproductive toxicity' in string or 'absence' in string or 'not' in string:
            type_ = 0
        
        elif 'reduce' in string or 'decrease' in string:
            type_ = 10
        
        elif 'LOAEL' in string or 'LOEC' in string or 'LO' in string:
            type_ = 1
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['target'] = df['Genotoxicity'].map(lambda x: effective(x))

# make for result
df = df[['CasRN','target']]

In [12]:
df.head()

CasRN  target
0  13674-84-5     0.0
1   3380-34-5     0.0
2    108-31-6     0.0
3     85-68-7     1.0
4     85-68-7     0.0

In [13]:
# save df
df.to_excel('tg475_raw.xlsx', header = True, index = False)

# smiles 추가

In [14]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [15]:
data = pd.read_excel('tg475_raw.xlsx')
len(data['CasRN'].unique())

387

In [16]:
data.head()

CasRN  target
0  13674-84-5     0.0
1   3380-34-5     0.0
2    108-31-6     0.0
3     85-68-7     1.0
4     85-68-7     0.0

In [17]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [18]:
data = data.drop_duplicates()
len(data)

383

In [19]:
tqdm.pandas()
data['SMILES'] = data.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|█████████████████████████████████████████████████████████████████████████████████| 383/383 [08:21<00:00,  1.31s/it]


In [20]:
data.SMILES.isna().sum()
data = data[data['SMILES'].notna()].reset_index(drop = True)

In [21]:
len(data)

265

In [22]:
data.to_excel('tg475.xlsx', header = True, index = False)